In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from audioDataLoader import audioDataloader
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import neuro
from py_apps import utils
from py_apps.utils.common_utils import read_network
from py_apps.utils.common_utils import load_json_arg
from py_apps.utils.neuro_help import *
import numpy as np


> ## Time Series Datasets
>
>classify_driver.py also supports timeseries data for classification. In this case, you will be using the app_type load and setting --timeseries true. <br>
>For datasets stored in this manner, if your data is stored in three-dimensional array A, then A[i] gives the first data  <br>
>instance, and A[i][j] will give the j<sup>th</sup> feature of the i<sup>th</sup> data instance, and A[i][j][k] gives the k<sup>th</sup> timestep of the j<sup>th</sup> feature of the i<sup>th</sup> data  <br>
>instance. An example of a time series data set is given in the data directory, the Activity Recognition system based on  <br>
>Multisensor data fusion (AReM) Data Set from the UCI repository. In this example, we are also using a custom encoder (with --encoder) and setting a  <br>
>new simulation time (with --sim_time), along with using multiple processes to train in parallel (with --processes).  <br>
<br>

## Summary 
<ul>
    <li>i: Index of a data instance. A[i] gives the first, A[i+1] the second, and so on.</li>
    <li>j: Feature index within a data instance. A[i][j] gives the j<sup>th</sup> feature.</li>
    <li>k: Timestep of a feature within a data instance. A[i][j][k] gets the i<sup>th</sup> data instance's j<sup>th</sup> feature's value at the k<sup>th</sup> timestep.</li>
</ul>
In summary:

    A[i] represents the ith data instance.
    A[i][j] represents the jth feature of the ith data instance.
    A[i][j][k] represents the kth timestep of the jth feature of the ith data instance.


## testing
You can test diffrent encoding paramiters with the <strong>spike_encoder_util</strong> from the framework repo. Utils programs are useful for testing and troubleshooting. 

<b>To make the util prgrames you will need to</b>

<ol>
    <li>checkout the framework repo </li>
    <li>Include the following flag in the makefile, positioned either under the line "Build Flags": <code>CFLAGS ?= -pthread</code></li>
    <li>run <strong>make utils</strong> from the root of the framework repo</li>
</ol>
 


You can then run any of the util programs from the root of the framework repo with <code>./bin/< < UTIL PROGRAM NAME > > '< < PROMPT > >'</code><br>
for example:<br>
<pre>
UNIX> pwd
/Users/USERNAME/Documents/framework
UNIX> <b>./bin/spike_encoder_tool 'SE>'</b>
SE> ?                                                                   # This prints out all of the options.
For commands that take a json either put a filename on the same line,
or the json can be multiple lines, starting on the next line.

FJ json                - Read a spike encoder from json.
TJ                     - Create JSON from the spike encoder.
PAS                    - Call print_all_settings() for debugging.
SOI int [over]         - Call set_overall_interval(interval, override=true).
I                      - Call get_overall_interval().
MS                     - Call get_max_spikes().
N                      - Return the number of neurons for the spike encoder.
GS val min max         - Get spikes for the given val, whose range is [min,max].
GTS vals min max       - Get timeseries spikes for the given vals, whose range is [min,max].
GSS vals times min max - Get sparse spikes for the given vals and times, and val range is [min,max] 

?                      - Print commands.
Q                      - Quit.
</pre>

In [4]:
train_dataset = "dataset_15sec/train/train.csv"
valid_dataset = "dataset_15sec/valid/valid.csv"
test_dataset = "dataset_15sec/test/test.csv"
data_transform = transforms.Compose([
        transforms.Normalize(mean=[2.3009], std=[42.1936]) 
    ])

train_data = audioDataloader(index_file=train_dataset, transforms=data_transform)
valid_data = audioDataloader(index_file=valid_dataset, transforms=data_transform)
test_data = audioDataloader(index_file=test_dataset, transforms=data_transform)

In [5]:
rotated_tensor = torch.transpose(train_data[0][0], 1, 2)

In [6]:
rotated_tensor.shape

torch.Size([1, 7201, 201])

In [7]:
data = torch.zeros([2,201,7201])
data

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [8]:
data[0] = train_data[0][0]
data[0]

tensor([[ 2.3665,  0.5570,  1.2286,  ..., -0.0420, -0.0333, -0.0505],
        [ 2.2995,  0.1478,  0.1014,  ..., -0.0409, -0.0432, -0.0512],
        [ 1.3559,  0.0761,  0.4851,  ..., -0.0503, -0.0539, -0.0542],
        ...,
        [-0.0543, -0.0545, -0.0545,  ..., -0.0545, -0.0545, -0.0545],
        [-0.0544, -0.0545, -0.0545,  ..., -0.0545, -0.0545, -0.0545],
        [-0.0544, -0.0545, -0.0545,  ..., -0.0545, -0.0545, -0.0545]])

In [9]:
data[1] = train_data[1][0]

In [10]:
data.shape

torch.Size([2, 201, 7201])

In [11]:
i = 0
data[i]

tensor([[ 2.3665,  0.5570,  1.2286,  ..., -0.0420, -0.0333, -0.0505],
        [ 2.2995,  0.1478,  0.1014,  ..., -0.0409, -0.0432, -0.0512],
        [ 1.3559,  0.0761,  0.4851,  ..., -0.0503, -0.0539, -0.0542],
        ...,
        [-0.0543, -0.0545, -0.0545,  ..., -0.0545, -0.0545, -0.0545],
        [-0.0544, -0.0545, -0.0545,  ..., -0.0545, -0.0545, -0.0545],
        [-0.0544, -0.0545, -0.0545,  ..., -0.0545, -0.0545, -0.0545]])

In [12]:
data[i].shape

torch.Size([201, 7201])

In [13]:
j = 0
data[i][j]

tensor([ 2.3665,  0.5570,  1.2286,  ..., -0.0420, -0.0333, -0.0505])

In [14]:
data[i][j].shape

torch.Size([7201])

In [15]:
k = 0
data.numpy()[i][j][k]

2.366456

    A[i] represents the ith data instance.
    A[i][j] represents the jth feature of the ith data instance.
    A[i][j][k] represents the kth timestep of the jth feature of the ith data instance.

In [17]:
print(f"there are {len(data)} data instance")
print(f"there are {len(data[i])} features of each data instance")
print(f"there are {len(data[i][j])} timestep in each feature of each data instance")


there are 2 data instance
there are 201 features of each data instance
there are 7201 timestep in each feature of each data instance


In [18]:
data = data.numpy()

In [20]:
data.shape

(2, 201, 7201)

In [21]:
train_dataset = "dataset_15sec/train/train.csv"
valid_dataset = "dataset_15sec/valid/valid.csv"
test_dataset = "dataset_15sec/test/test.csv"
data_transform = transforms.Compose([
        transforms.Normalize(mean=[2.3009], std=[42.1936]) 
    ])

train_data = audioDataloader(index_file=train_dataset, transforms=data_transform)
valid_data = audioDataloader(index_file=valid_dataset, transforms=data_transform)
test_data = audioDataloader(index_file=test_dataset, transforms=data_transform)

In [22]:
data_instance_dim = len(train_data)
data_instance_dim

40000

In [23]:
features_dim = len(train_data[0][0][0])
features_dim

201

In [24]:
timestep_dim = len(train_data[0][0][0][0])
timestep_dim

7201

In [28]:
train_data[0][0].shape

torch.Size([1, 201, 7201])